In [1]:
import json
import pandas as pd
import requests
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100
pd.set_option('display.max_colwidth', -1)

/var/folders/st/8jn3gyls0_nbxy23bw4zmds00000gn/T/ipykernel_95234/2703078761.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
def getDataframe(filename):
    arr = []
    with open('./data/' + filename) as out:
        for line in out:
            j_content = json.loads(line)
            arr.append(j_content)
    df = pd.DataFrame(arr)
    return df

In [19]:
def mongoIdToId(value):
    return dict(value)["$oid"]

In [37]:
producers = getDataframe('producers.json')
producers["organization_id"] = producers["associatedTo"].apply(mongoIdToId)
producers.head()

,_id,associatedTo,__v,organization_id
0,AFB,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123
1,AFB - DR Nouvelle-Aquitaine,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123
2,ALTOA,{'$oid': '580fb83a88ee38771813e4cb'},NaN,580fb83a88ee38771813e4cb
3,ANFR,{'$oid': '54256fe588ee380af6717348'},NaN,54256fe588ee380af6717348
4,ARS,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123


In [44]:
organizations = getDataframe('organizations.json')
organizations["geop_organization_id"] = organizations["_id"].apply(mongoIdToId)
organizations = organizations.merge(producers, left_on="geop_organization_id", right_on="organization_id")
organizations["geop_organization_name"] = organizations["_id_y"]
organizations = organizations[["geop_organization_id", "geop_organization_name"]]
organizations.head()

,geop_organization_id,geop_organization_name
0,534fff68a3a7292c64a77d8d,Département de Saône-et-Loire
1,534fff68a3a7292c64a77d8d,Département de Saône-et-Loire (71)
2,534fff6aa3a7292c64a77d96,Région Hauts-de-France
3,534fff6ba3a7292c64a77da1,ASPA
4,534fff6ba3a7292c64a77da1,CConseil Départemental du Bas-Rhin (CD67)


In [45]:
organizations['dgv_organization_id'] = organizations['geop_organization_id']

In [46]:
dgv_catalog_orgas = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/b7bbfedc-2448-4135-a6c7-104548d396e7',sep=';')

In [47]:
dgv_catalog_orgas = dgv_catalog_orgas[['id','name']]

In [48]:
dgv_catalog_orgas = dgv_catalog_orgas.rename(columns={'id': 'dgv_organization_id','name':'dgv_organization_name'})

In [49]:
organizations = organizations.drop_duplicates(keep="first")
dgv_catalog_orgas = dgv_catalog_orgas.drop_duplicates(keep="first")

In [50]:
organizations.columns

Index(['geop_organization_id', 'geop_organization_name',
       'dgv_organization_id'],
      dtype='object')

In [51]:
dgv_catalog_orgas.dgv_organization_id.nunique()

3222

In [60]:
final_orgas = dgv_catalog_orgas.merge(organizations, on='dgv_organization_id', how='inner')
final_orgas.head()

,dgv_organization_id,dgv_organization_name,geop_organization_id,geop_organization_name
0,5e44026b8b4c416f93430c66,Colmar Agglomération,5e44026b8b4c416f93430c66,Colmar Agglomération
1,5d823fd98b4c411e38e820b4,Fluo Grand Est,5d823fd98b4c411e38e820b4,Communauté Urbaine du Grand Reims
2,5d823fd98b4c411e38e820b4,Fluo Grand Est,5d823fd98b4c411e38e820b4,Communauté d'Agglomération Ardenne Métropole
3,5d823fd98b4c411e38e820b4,Fluo Grand Est,5d823fd98b4c411e38e820b4,Communauté d'Agglomération Bar-le-Duc Sud Meuse
4,5d823fd98b4c411e38e820b4,Fluo Grand Est,5d823fd98b4c411e38e820b4,Communauté d'Agglomération Epernay Coteaux et Plaine de Champagne


In [62]:
geoc = pd.read_excel('./data/Liste_catalogues_DCAT.xlsx')

In [63]:
geoc = geoc.rename(columns={
    'Producteurs': 'geoc_organization_name', 
    'uuid': 'geoc_organization_uuid', 
    'url de moissonnage ': 'geoc_organization_url'
})

In [65]:
dff = final_orgas
dff['geoc_organization_name'] = None
dff['geoc_organization_uuid'] = None
dff['geoc_organization_url'] = None

In [66]:
def removeCommonWords(org):
    org = str(org)
    org = org.replace(' de ',' ').replace(' un ',' ').replace(' à ','').replace(' l\'',' ').replace(' d\'', ' ')
    org = org.replace(' des ',' ').replace(' pour ', ' ').replace(' les ', ' ').replace('(','').replace(')','')
    org = org.replace(' - ',' ').replace(',','').replace(' du ', ' ').replace('à','a').replace('é','e')
    org = org.replace('ê','e').replace('è','e').replace('\'',' ')
    return org.lower()

In [71]:
arr = []
cpt = 0
for index, row in dff.iterrows():
    for index2, row2 in geoc.iterrows():
        if(row['geop_organization_name'] == row['geop_organization_name']):
            if removeCommonWords(row['geop_organization_name']) in removeCommonWords(row2['geoc_organization_name']):
                row['geoc_organization_name'] = row2['geoc_organization_name']
                row['geoc_organization_uuid'] = row2['geoc_organization_uuid']
                row['geoc_organization_url'] = row2['geoc_organization_url']
            elif(row['dgv_organization_name'] == row['dgv_organization_name']):
                if removeCommonWords(row['dgv_organization_name']) in removeCommonWords(row2['geoc_organization_name']):
                    row['geoc_organization_name'] = row2['geoc_organization_name']
                    row['geoc_organization_uuid'] = row2['geoc_organization_uuid']
                    row['geoc_organization_url'] = row2['geoc_organization_url']

    arr.append(row)

In [72]:
df_final = pd.DataFrame(arr)

In [73]:
df_final.shape

(551, 7)

In [86]:
df_final[df_final['geoc_organization_uuid'].notna()]["geoc_organization_url"].drop_duplicates().shape

(125,)

In [75]:
df_final.to_csv('mapping_geop_dgv_geoc.csv',index=False)